In [1]:
import numpy as np
# import tensorflow as tf
from tensorflow import keras
# import tensorflow
from tensorflow.keras.layers import BatchNormalization, Dense, Input, BatchNormalization
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GaussianNoise


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import GaussianNoise

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, GaussianNoise, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

2023-10-22 10:51:19.398301: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Convolution2D, Activation, Flatten, MaxPooling2D,Input,Dropout,GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.framework import ops
from binarization_utils import *

In [3]:
resid_levels = True
BINARY = False
LOGIC_SHRINKAGE = False
custom_rand_seed = 1

In [4]:
import os
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [5]:
def autoenco(k,n,SNRs):
    BER = []
    R = float(k/n)
    Nt = 10000
    M = 2**k


    label = np.random.randint(M,size=Nt)
    data = [] # one-hot encodedno


    for i in label:
        temp = np.zeros(M)
        temp[i] = 1
        data.append(temp) 


    data = np.array(data)


#     autoencoder = Sequential()
#     autoencoder.add(Dense(M, activation='relu', input_shape=(M,), name='input_layer'))
#     autoencoder.add(Dense(n, activation='linear', name='encoder_layer_1'))
#     autoencoder.add(BatchNormalization(name='encoder_batchnorm_layer'))
    
    
    autoencoder = Sequential()
    autoencoder.add(binary_dense(levels=resid_levels, n_in=M, n_out=M, input_shape=(M,), BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="input_layer"))
    autoencoder.add(Activation('relu'))
    
    autoencoder.add(binary_dense(levels=resid_levels, n_in=M, n_out=n, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="encoder_layer_1"))
    autoencoder.add(Activation('linear'))
    
    autoencoder.add(BatchNormalization(name='encoder_batchnorm_layer'))

    
    
    
    EbNo_train = 10 ** (0.7)
    noise_value = np.sqrt(1/(2*R*EbNo_train))
    
    
    autoencoder.add(GaussianNoise(noise_value))
    
    
#     autoencoder.add(Dense(M, activation='relu', name='decoder_layer_1'))
#     autoencoder.add(Dense(M, activation='softmax', name='decoder_layer_2'))
#     #sgd = SGD(lr=0.001)
#     autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
    
    
    
    autoencoder.add(binary_dense(levels=resid_levels, n_in=n, n_out=M, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="decoder_layer_1"))
    autoencoder.add(Activation('relu'))
    
    autoencoder.add(binary_dense(levels=resid_levels, n_in=M, n_out=M, BINARY=BINARY, LOGIC_SHRINKAGE=LOGIC_SHRINKAGE, custom_rand_seed=custom_rand_seed, name="decoder_layer_2"))
    autoencoder.add(Activation('softmax'))
    autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
    
    
    print(autoencoder.summary())
    
    
    
    N_val = 1500
    val_label = np.random.randint(M,size=N_val)
    val_data = []
    
    for i in val_label:
        temp = np.zeros(M)
        temp[i] = 1
        val_data.append(temp)
    
    val_data = np.array(val_data)
    
    


    autoencoder.fit(data, data,epochs=30,
                  batch_size=300,
                  validation_data=(val_data, val_data),
                  callbacks=[ModelCheckpoint(save_best_only=True, 
                                             filepath="ok")])
 
    
    



#     encoder = Sequential()
#     encoder.add(BatchNormalization(input_shape=(M,), name='encoder_2_input_layer_with_batchnorm'))
#     encoded_input = Input(shape=(n,))
    


    return BER

In [6]:
# use k =3 and n = 7
SNRs = np.arange(1.0,18.0,0.5)

autoenco(3, 7, SNRs)


	def call(self, x,mask=None):

		#np.random.seed(self.custom_rand_seed)
		#tf.set_random_seed(self.rand_seed)
		#random.seed(self.rand_seed)

		constraint_gamma=K.abs(self.gamma)#K.clip(self.gamma,0.01,10)
		if self.levels==1 or self.first_layer==True:
			if self.BINARY==False:
				self.clamped_w=constraint_gamma*K.clip(self.w,-1,1)
			else:
				self.clamped_w=constraint_gamma*binarize(self.w)
			self.out=K.dot(x,self.clamped_w)
		elif self.levels==2:
			if self.LUT==True:

				if self.LOGIC_SHRINKAGE==True:

					#SHRINKAGE
                #Stacking c coef	ficients to create c_mat1 and c_mat2 tensors
					c_mat1=self.c_param[0 : 2**self.k_lut, :, :]
					c_mat2=self.c_param[2**self.k_lut : (2**self.k_lut)*2, :, :]

					c_mat1=tf.reshape(c_mat1,(1,2**self.k_lut,self.n_in*self.n_out))                
					c_mat2=tf.reshape(c_mat2,(1,2**self.k_lut,self.n_in*self.n_out))
       
					c_mat1=tf.transpose(c_mat1,(2,0,1))
					c_mat2=tf.transpose(c_mat2,(2,0,1))
            
					c_mat1=tf

2023-10-22 10:51:22.662209: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-10-22 10:51:23.147519: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-22 10:51:23.147600: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 5310f4639566
2023-10-22 10:51:23.147608: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 5310f4639566
2023-10-22 10:51:23.147731: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.82.1
2023-10-22 10:51:23.147764: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.82.1
2023-10-22 10:51:23.147771: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.82.1
2023-10-22 10:51:23.162276: I tensorflow/core/platform/pro

10000/10000 [==============================] - 1s 115us/sample - loss: 1.9585 - val_loss: 2.0243
Epoch 2/30
10000/10000 [==============================] - 0s 8us/sample - loss: 1.6894 - val_loss: 1.9379
Epoch 3/30
10000/10000 [==============================] - 0s 8us/sample - loss: 1.4293 - val_loss: 1.8007
Epoch 4/30
10000/10000 [==============================] - 0s 8us/sample - loss: 1.1458 - val_loss: 1.5976
Epoch 5/30
10000/10000 [==============================] - 0s 8us/sample - loss: 0.8774 - val_loss: 1.3309
Epoch 6/30
10000/10000 [==============================] - 0s 8us/sample - loss: 0.6662 - val_loss: 1.0352
Epoch 7/30
10000/10000 [==============================] - 0s 8us/sample - loss: 0.5010 - val_loss: 0.7546
Epoch 8/30
10000/10000 [==============================] - 0s 8us/sample - loss: 0.3583 - val_loss: 0.5014
Epoch 9/30
10000/10000 [==============================] - 0s 8us/sample - loss: 0.2186 - val_loss: 0.2916
Epoch 10/30
10000/10000 [==============================

[]